# core

> Module containing the reading and conversion logic from the sl2 and sl3 files to csv and shape

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

We will use [sonarlight](https://github.com/KennethTM/sonarlight) to read the measurements from the `sl2` or `sl3` files.

::: {.callout-note}
Previously we used [sslib](https://github.com/opensounder/python-sllib) to parse the sonar files. But this package latest commit was 4 years ago. This `sslib` package also has less stars and the `sonarlight` has some neat extra features, such as simple conversion to a Pandas dataframe.
:::

In [ ]:
#| export
import geopandas as gpd
from pathlib import Path
from sonarlight import Sonar

In [ ]:
sl2file = Path("/app/data/sonar2csv_shape/test/2024-07-11_zuiderpark Hoogeveen2_+1075cmnap.sl2")
sl3file = Path("/app/data/sonar2csv_shape/test/Sonar_2022-04-26_21.07.11beschrijving+0765cmNAP.sl3")

In [ ]:
type(sl2file)

pathlib.PosixPath

How should I do type anotating for `sl2file`?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

For type annotating `sl2file`, you can use `Path` from the `pathlib` module:

```python
sl2file: Path = Path("/app/data/sonar2csv_shape/test/2024-07-11_zuiderpark Hoogeveen2_+1075cmnap.sl2")
```

Since you've already imported `Path` from `pathlib`, this will work perfectly. The type annotation `Path` covers both `PosixPath` (on Unix/Linux/Mac) and `WindowsPath` (on Windows), as these are subclasses of `Path`.

Would you like to know about any other type annotation options for file paths?

In [ ]:
#| export
def read_sl(
    filepath: Path # The absolute location of the file to convert
    )->Sonar:
    return Sonar(str(filepath))

In [ ]:
sl2data = read_sl(sl2file)

In [ ]:
sl2data

Summary of SL2 file:

- Primary channel with 2963 frames
- Secondary channel with 2963 frames
- Downscan channel with 2963 frames
- Sidescan channel with 2962 frames

Start time: 2024-07-11 08:13:50.342999935
End time: 2024-07-11 08:17:13.848999977

File info: version 2, device 2, blocksize 3200, frame version 0

In [ ]:
type(sl2data)

sonarlight.sonar_class.Sonar

In [ ]:
sl2data.df

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
1,0,primary,2024-07-11 08:13:50.342999935,718716,6906589,6.478055,52.711570,0.00000,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
4,0,secondary,2024-07-11 08:13:50.344000101,718716,6906589,6.478055,52.711570,0.00000,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
7,0,downscan,2024-07-11 08:13:50.404000044,718714,6906589,6.478037,52.711570,0.00000,2.40792,0.765342,0.118687,0.679109,12.19,444,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
8,1,sidescan,2024-07-11 08:13:50.404999971,718714,6906589,6.478037,52.711570,-4.99872,4.99872,0.765342,0.118687,0.679109,12.19,214,"[41, 40, 40, 42, 42, 44, 47, 48, 48, 48, 48, 4..."
9,1,primary,2024-07-11 08:13:50.385999918,718716,6906589,6.478055,52.711570,0.00000,3.99288,0.765342,0.118292,0.683147,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25276,2972,downscan,2024-07-11 08:17:13.760999918,718672,6906534,6.477659,52.711270,0.00000,2.40792,0.924873,0.615444,5.473728,12.34,537,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
25277,4466,sidescan,2024-07-11 08:17:13.760999918,718672,6906534,6.477659,52.711270,-4.99872,4.99872,0.924873,0.615444,5.473728,12.34,259,"[45, 47, 48, 48, 48, 48, 47, 46, 46, 46, 46, 4..."
25279,2973,primary,2024-07-11 08:17:13.786999941,718673,6906533,6.477668,52.711264,0.00000,3.99288,0.941796,0.599854,5.472764,12.34,724,"[214, 214, 214, 214, 214, 214, 214, 214, 214, ..."
25282,2973,secondary,2024-07-11 08:17:13.788000107,718673,6906533,6.477668,52.711264,0.00000,3.99288,0.941796,0.599854,5.472764,12.34,724,"[214, 214, 214, 214, 214, 214, 214, 214, 214, ..."


We need to do several processing steps to go from the `sl2` or `sl3` data to a `csv` and `shape` we can use in our GIS software.

**From depth in meter to bottom height in mNAP**

To convert from depth measurement in meter to bottom height measurement in mNAP we need to extract the height in mNAP from which the depth measurements were taken. Since our Sonar boot doesn't measure it's own height, we add this height manually in the filename. We add it add the end of filename, some examples:

- `2024-07-11_zuiderpark Hoogeveen2_+1075cmnap.sl2`
- `Sonar_2022-04-26_21.07.11beschrijving+0765cmNAP.sl3`

**Filter relevant facts**

We only need the facts that have the value "primary" in the column "survey".

**Transformation to the correct CRS**

The coördinates in the Sonar files are in crs WGS84 (epsg:4326), we need to convert those to the crs we use, which is RDN Amersfoort (epsg:28992).
We accomplish that by using the `geopandas` method `.set_crs` and `.to_crs`.

In [ ]:
sl2df_sml = sl2data.df[0:16]
sl2df_sml

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
1,0,primary,2024-07-11 08:13:50.342999935,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
4,0,secondary,2024-07-11 08:13:50.344000101,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
7,0,downscan,2024-07-11 08:13:50.404000044,718714,6906589,6.478037,52.71157,0.00000,2.40792,0.765342,0.118687,0.679109,12.19,444,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
8,1,sidescan,2024-07-11 08:13:50.404999971,718714,6906589,6.478037,52.71157,-4.99872,4.99872,0.765342,0.118687,0.679109,12.19,214,"[41, 40, 40, 42, 42, 44, 47, 48, 48, 48, 48, 4..."
9,1,primary,2024-07-11 08:13:50.385999918,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118292,0.683147,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
12,1,secondary,2024-07-11 08:13:50.387000084,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118292,0.683147,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
15,1,downscan,2024-07-11 08:13:50.447000027,718714,6906589,6.478037,52.71157,0.00000,2.40792,0.765342,0.118292,0.683147,12.19,444,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
16,2,sidescan,2024-07-11 08:13:50.447999954,718714,6906589,6.478037,52.71157,-4.99872,4.99872,0.765342,0.118292,0.683147,12.19,214,"[40, 47, 52, 55, 55, 57, 58, 57, 54, 54, 48, 4..."
18,2,primary,2024-07-11 08:13:50.473000050,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.763347,0.117554,0.690681,12.19,587,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
21,2,secondary,2024-07-11 08:13:50.473999977,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.763347,0.117554,0.690681,12.19,587,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."


In [ ]:
sl2df_sml.info

<bound method DataFrame.info of     id     survey                      datetime       x        y  longitude  \
1    0    primary 2024-07-11 08:13:50.342999935  718716  6906589   6.478055   
4    0  secondary 2024-07-11 08:13:50.344000101  718716  6906589   6.478055   
7    0   downscan 2024-07-11 08:13:50.404000044  718714  6906589   6.478037   
8    1   sidescan 2024-07-11 08:13:50.404999971  718714  6906589   6.478037   
9    1    primary 2024-07-11 08:13:50.385999918  718716  6906589   6.478055   
12   1  secondary 2024-07-11 08:13:50.387000084  718716  6906589   6.478055   
15   1   downscan 2024-07-11 08:13:50.447000027  718714  6906589   6.478037   
16   2   sidescan 2024-07-11 08:13:50.447999954  718714  6906589   6.478037   
18   2    primary 2024-07-11 08:13:50.473000050  718716  6906589   6.478055   
21   2  secondary 2024-07-11 08:13:50.473999977  718716  6906589   6.478055   
24   2   downscan 2024-07-11 08:13:50.535000086  718714  6906589   6.478037   
25   4   sidescan 20

The `sl2df_sml` dataframe also has a column `gps_altitude`. Can you explain what this probably is. Given that longitude and latitude are given in the WGS84 coordinate system?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! The `gps_altitude` column is likely the **altitude (elevation) above the WGS84 ellipsoid**, measured in meters. 

In the WGS84 coordinate system, GPS devices typically provide three coordinates:
- Longitude and latitude (horizontal position)
- Altitude/elevation (vertical position)

This GPS altitude is usually the height above the WGS84 reference ellipsoid (a mathematical model of Earth's shape), not the height above sea level or above a local datum like NAP.

Looking at your data, the `gps_altitude` values are around 12.19 meters. This would be the height of the GPS antenna (on your sonar boat) above the WGS84 ellipsoid at that location.

Is this different from the height information you're extracting from the filename (like "+1075cmnap")? What do you think the relationship might be between these two height measurements?

Great. How can I transform this `gps_altitude` to mNAP as used in the RDnew Amersfoort system (epsg:28992)?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking! However, there's an important distinction to make here:

**GPS altitude and coordinate system transformations work differently than horizontal coordinates.**

When you transform longitude/latitude from WGS84 (epsg:4326) to RD New (epsg:28992) using `.to_crs()`, it handles the horizontal position transformation well. But the vertical component (altitude) is trickier because:

1. GPS altitude is height above the WGS84 ellipsoid
2. NAP (Normaal Amsterdams Peil) is a height above mean sea level
3. These two reference surfaces don't have a simple constant offset - the difference varies by location

To properly convert GPS altitude to mNAP, you typically need a **geoid model** or a transformation grid specific to the Netherlands (like NLGEO2018 or similar). This accounts for the local variations between the ellipsoid and the geoid (mean sea level).

Do you have access to such a transformation tool, or is there a reason you're interested in converting the GPS altitude? I notice you're already extracting the NAP height from the filename - is that serving a different purpose?

It would be good if we could skip the NAP height from the filename. Since that is a manual proces. Which may give errors or be forgotten when the file is uploaded.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That makes sense - automating this would definitely reduce errors and make the workflow more robust!

To convert GPS altitude (WGS84 ellipsoid height) to mNAP, you'll need to apply a geoid correction. In the Netherlands, the standard approach is to use the **RDNAPTRANS** transformation, which includes both horizontal (RD) and vertical (NAP) transformations.

Python libraries that can handle this include:
- **pyproj** - can handle vertical transformations with the right geoid model
- **PROJ** (the underlying library) - supports compound CRS transformations

The key is to use a compound coordinate reference system that includes both horizontal and vertical components. For the Netherlands, you'd want something like EPSG:7415 (RD New + NAP height).

Have you used `pyproj` before, or would you like to explore how to set up this transformation? It will involve a few steps beyond the simple `.to_crs()` you're currently using.

How to convert the pandas dataframe `sl2df_sml` to a geodataframe? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To convert your pandas DataFrame `sl2df_sml` to a GeoDataFrame, you need to create geometry objects from your coordinate columns. Since you have `longitude` and `latitude` columns, you can use those to create Point geometries.

Here's how you'd approach it:

```python
gdf = geopandas.GeoDataFrame(sl2df_sml, geometry=geopandas.points_from_xy(sl2df_sml.longitude, sl2df_sml.latitude))
```

This creates Point geometries from your longitude/latitude columns. However, you'll also want to set the coordinate reference system (CRS) since these are WGS84 coordinates. Would you like to try writing that part yourself, or would you like a hint about which method to use?

In [ ]:
gdf = gpd.GeoDataFrame(sl2df_sml, geometry=gpd.points_from_xy(sl2df_sml.longitude, sl2df_sml.latitude))

In [ ]:
gdf = gdf.set_crs(epsg=4326)

In [ ]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
gdf = gdf.to_crs(epsg=28992)

In [ ]:
gdf.crs

<Projected CRS: EPSG:28992>
Name: Amersfoort / RD New
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Netherlands - onshore, including Waddenzee, Dutch Wadden Islands and 12-mile offshore coastal zone.
- bounds: (3.2, 50.75, 7.22, 53.7)
Coordinate Operation:
- name: RD New
- method: Oblique Stereographic
Datum: Amersfoort
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

I want to build a regex to extract the height in all of the following cases:

`measurements_-720cmnap.sl2`
`measurements_+720cmNap.sl3`
`measurements_-1720cmnap.sl3`
`measurements-1720cmnap.sl3`
`measurements+20cmNAP.sl2`
`measurements+20CMNAP.sl2`

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! You want to extract the height value (including the sign) from those filenames. Let's break down what patterns you need to match:

- Either a leading `+` or `-` sign
- One or more digits
- Followed by `cmnap` (case-insensitive)

You only need the number with its sign.

Correct. Please explain how I do this.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To extract this pattern with regex, you'll need to use Python's `re` module. Here's the approach:

**Pattern components:**
- `[+-]` matches either + or -
- `\d+` matches one or more digits
- `(?=cmnap)` is a lookahead that checks for "cmnap" (case-insensitive) without including it in the match

You'll want to use `re.search()` to find the pattern in the filename string, and use the `re.IGNORECASE` flag to handle different capitalizations.

Would you like to try writing the regex pattern yourself, or would you like me to show you an example?

In [ ]:
import re

In [ ]:
rgx = r"[+-]\d+(?=cmnap)"

In [ ]:
int(re.search(rgx, str(sl2file), flags=re.IGNORECASE)[0])

1075

In [ ]:
#| export
def extract_height(
    sl_filepath: Path, # The absolute location of the file to convert
    re_ptrn: str=r"[+-]\d+(?=cmnap)"
    )->int: # The height of the measurement station in cm above NAP
    "Extract height from the filename in cmNAP"
    if 'cmnap' not in str(sl_filepath).lower():
        raise ValueError("The filename must contain the height of the Sonar boot at time of measurement in 'cmNAP' at the end of the filename (e.g. 'example_description_+1050cmNAP.sl2')")
    return int(re.search(re_ptrn, str(sl2file), flags=re.IGNORECASE[0]))

In [ ]:
def sl2gdf(
    sl_filepath: Path, # The absolute location of the file to convert
    msrmnt_height: int, # Height of the measurement instrument at time of taking the measurements
    to_crs: str = "epsg:28992", # epsg code of crs to transform the coördinates to
    survey_fltr: str = "primary", # Filter measurement facts on survey value
    )->gpd:
    "Convert a sl2 or sl3 file to a GeoDataFrame with the given crs."
    s = Sonar(str(sl_filepath))
    df = s.df
    gdf = gpd.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
    gdf = gdf.set_crs(epsg=4326)
    return gdf.to_crs(to_crs)

In [ ]:
"water_depth" in gdf.columns

True

In [ ]:
sl2df_sml.head()

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
1,0,primary,2024-07-11 08:13:50.342999935,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
4,0,secondary,2024-07-11 08:13:50.344000101,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."
7,0,downscan,2024-07-11 08:13:50.404000044,718714,6906589,6.478037,52.71157,0.00000,2.40792,0.765342,0.118687,0.679109,12.19,444,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
8,1,sidescan,2024-07-11 08:13:50.404999971,718714,6906589,6.478037,52.71157,-4.99872,4.99872,0.765342,0.118687,0.679109,12.19,214,"[41, 40, 40, 42, 42, 44, 47, 48, 48, 48, 48, 4..."
9,1,primary,2024-07-11 08:13:50.385999918,718716,6906589,6.478055,52.71157,0.00000,3.99288,0.765342,0.118292,0.683147,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ..."


In [ ]:
sl2df_sml_pr = sl2df_sml[sl2df_sml["survey"]=="primary"]

In [ ]:
sl2df_sml_pr['bottom_height'] = 12.53/100 - sl2df_sml_pr['water_depth']

/tmp/ipykernel_46/651093007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sl2df_sml_pr['bottom_height'] = 12.53/100 - sl2df_sml_pr['water_depth']


Please explain the DataFrame warning.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

This warning occurs because of how you created `sl2df_sml_pr`. When you filtered the dataframe with:

```python
sl2df_sml_pr = sl2df_sml[sl2df_sml["survey"]=="primary"]
```

Pandas doesn't know if this created a **copy** of the data or just a **view** (reference) to the original data. When you then try to add a new column to `sl2df_sml_pr`, pandas warns you that changes might unexpectedly affect the original `sl2df_sml` dataframe (or might not work at all).

**The solution:** Use `.copy()` to explicitly create a new dataframe:

```python
sl2df_sml_pr = sl2df_sml[sl2df_sml["survey"]=="primary"].copy()
```

Or use `.loc[]` for the assignment as the warning suggests. 

Would you like to try fixing this yourself, or do you have questions about why this happens?

In [ ]:
sl2df_sml_pr = sl2df_sml[sl2df_sml["survey"]=="primary"].copy()

In [ ]:
sl2df_sml_pr['bottom_height'] = 12.53/100 - sl2df_sml_pr['water_depth']

In [ ]:
sl2df_sml_pr.head()

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames,bottom_height
1,0,primary,2024-07-11 08:13:50.342999935,718716,6906589,6.478055,52.71157,0.0,3.99288,0.765342,0.118687,0.679109,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ...",-0.640042
9,1,primary,2024-07-11 08:13:50.385999918,718716,6906589,6.478055,52.71157,0.0,3.99288,0.765342,0.118292,0.683147,12.19,588,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ...",-0.640042
18,2,primary,2024-07-11 08:13:50.473000050,718716,6906589,6.478055,52.71157,0.0,3.99288,0.763347,0.117554,0.690681,12.19,587,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ...",-0.638047
26,3,primary,2024-07-11 08:13:50.519000052,718716,6906589,6.478055,52.71157,0.0,3.99288,0.763347,0.117210,0.700539,12.19,587,"[210, 210, 210, 210, 210, 210, 210, 210, 210, ...",-0.638047


I think that all measurements with the same "bottom_index" also have the same longitude and latitude and water_depth. How can I check that assumption?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good hypothesis! To check this, you could group by `bottom_index` and see if the values within each group are all the same.

One approach is to use `.groupby()` with `.nunique()` (number of unique values). If a column has only 1 unique value per group, then all values in that group are identical.

Try something like:

```python
sl2df_sml_pr.groupby('bottom_index')[['longitude', 'latitude', 'water_depth']].nunique()
```

This will show you how many unique values exist for each column within each `bottom_index` group. What do you expect to see if your assumption is correct?

In [ ]:
sl2data.df[sl2data.df['survey']=="primary"].groupby('bottom_index')[['longitude', 'latitude', 'water_depth']].nunique()

,longitude,latitude,water_depth
bottom_index,,,
473,1,1,1
475,2,2,1
476,1,1,1
478,1,1,1
479,1,1,1
...,...,...,...
808,1,1,1
809,1,1,1
810,1,1,1


In [ ]:
sl2data.df[sl2data.df["bottom_index"]==475]

,id,survey,datetime,x,y,longitude,latitude,min_range,max_range,water_depth,gps_speed,gps_heading,gps_altitude,bottom_index,frames
4126,485,downscan,2024-07-11 08:14:23.638000011,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818000,0.771551,4.662271,13.450000,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4134,486,downscan,2024-07-11 08:14:23.678999901,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818000,0.767841,4.660583,13.450000,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4177,491,downscan,2024-07-11 08:14:24.032000065,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.743761,4.644800,13.450000,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4185,492,downscan,2024-07-11 08:14:24.076999903,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.741315,4.641526,13.450000,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4211,495,downscan,2024-07-11 08:14:24.296000004,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.732592,4.624510,13.339999,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4219,496,downscan,2024-07-11 08:14:24.342000008,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.730653,4.619778,13.339999,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4228,497,downscan,2024-07-11 08:14:24.430999994,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.729701,4.607848,13.339999,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4236,498,downscan,2024-07-11 08:14:24.471999884,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.729258,4.600700,13.339999,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4245,499,downscan,2024-07-11 08:14:24.559999943,718673,6906583,6.477668,52.711537,0.0,2.40792,0.818029,0.728834,4.593876,13.339999,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."
4253,500,downscan,2024-07-11 08:14:24.805999994,718672,6906583,6.477659,52.711537,0.0,2.40792,0.818029,0.724261,4.552243,13.339999,475,"[228, 228, 228, 228, 228, 228, 228, 228, 228, ..."


It is mostly true that the same bottom_index is the same location and depth. But not always. So we must filter the resulting table on same locations. Could we use something like `.unique` to only keep those points that have a unique `longitude`, `latitude` combination?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, good thinking! You can use `.drop_duplicates()` to keep only unique combinations of longitude and latitude.

The method would look like:

```python
df_unique = df.drop_duplicates(subset=['longitude', 'latitude'])
```

This keeps the first occurrence of each unique longitude/latitude pair and removes the rest.

Would you like to try applying this to your filtered primary survey data? Or do you have questions about how `drop_duplicates()` works?

In [ ]:
sl2df_sml_pr['bottom_height'] = 12.53/100 - sl2df_sml_pr['water_depth']

/tmp/ipykernel_46/651093007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sl2df_sml_pr['bottom_height'] = 12.53/100 - sl2df_sml_pr['water_depth']


Great. I lost my train of thought. We are building a function that accepts the GeoDataframe from the function `sl2gdf` and an integer wich is the msrmnt_height: int, # Height of the measurement instrument at time of taking the measurements. The function filter the dataframe on "survey"=="primary", and remove duplicates of ("longitude", "latitude", "water_depth"). Then it should create a new column "bottom_height" for which the values can be calculated by `msrmnt_height/100 - water_depth`. All parts of this function we tested above. Am I missing something? Can you write this function based on the tests above? The result should be a new GeoDataframe. We can name the function: clean_gdf.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Based on your tests above, here's the `clean_gdf` function:

```python
def clean_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame from sl2gdf
    msrmnt_height: int, # Height of measurement instrument in cm above NAP
    ) -> gpd.GeoDataFrame: # Cleaned GeoDataFrame with bottom_height column
    "Filter primary survey data, remove duplicates, and calculate bottom height in mNAP"
    gdf_primary = gdf[gdf["survey"] == "primary"].copy()
    gdf_unique = gdf_primary.drop_duplicates(subset=['longitude', 'latitude', 'water_depth'])
    gdf_unique['bottom_height'] = msrmnt_height / 100 - gdf_unique['water_depth']
    return gdf_unique
```

Does this capture everything you need, or would you like to adjust anything?

In [ ]:
#| export
def clean_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame from sl2gdf
    msrmnt_height: int, # Height of measurement instrument in cm above NAP
    ) -> gpd.GeoDataFrame: # Cleaned GeoDataFrame with bottom_height column
    "Filter primary survey data, remove duplicates, and calculate bottom height in mNAP"
    gdf_primary = gdf[gdf["survey"] == "primary"].copy()
    gdf_unique = gdf_primary.drop_duplicates(subset=['longitude', 'latitude', 'water_depth'])
    gdf_unique['bottom_height'] = msrmnt_height / 100 - gdf_unique['water_depth']
    return gdf_unique

I also would like to check if the Dataframe that is created from the sl2 or sl2 file contains the needed columns in the needed datatype. Should I write a function to check that? Should I add the checks within an existing function? Should I use Pydantic?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question about validation! You have a few options:

1. **Separate validation function** - keeps concerns separated, reusable
2. **Add checks within existing functions** - catches issues early where they matter
3. **Use Pydantic** - more robust, but adds dependency and complexity

For your case, I'd suggest starting simple: add basic checks within your existing functions (like `sl2gdf` or `clean_gdf`) to verify the required columns exist. Something like:

```python
required_cols = ['longitude', 'latitude', 'water_depth', 'survey']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Missing required columns...")
```

Pydantic would be great if you're building a larger system with complex validation needs, but might be overkill here.

Which approach feels right for your use case? Are you expecting the sonar file format to be stable, or do you need robust validation?

In [ ]:
#| export
def sl2gdf(
    sl_filepath: Path, # The absolute location of the file to convert
    to_crs: str = "epsg:28992", # epsg code of crs to transform the coördinates to
    survey_fltr: str = "primary", # Filter measurement facts on survey value
    )->gpd:
    "Convert a sl2 or sl3 file to a GeoDataFrame with the given crs."
    s = Sonar(str(sl_filepath))
    df = s.df
    required_cols = ['longitude', 'latitude', 'water_depth', 'survey']
    if not all(col in df.columns for col in required_cols):
        raise KeyError(f"Missing one or more of the required columns in the converted sl2 or sl3 file.\nRequired columns are: 'longitude', 'latitude', 'water_depth' and 'survey'")
    gdf = gpd.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))
    gdf = gdf.set_crs(epsg=4326)
    return gdf.to_crs(to_crs)

I want to export a  GeoDataframe to different filetypes. See `export_gdf` function. How do I do that?

def export_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame to be saved
    fn: str, # Filename of the GeoDataFrame without extension
    folder_out: Path, # Absolute path to folder where files can be saved
    esri_shp: bool=True, # Save GeoDataFrame to Esri shapefile?
    csv: bool=True, # Save GeoDataFrame to comma seperated file?
    geopckg: bool=True, # Save GeoDataFrame to geopackage?
    )->None:
    if esri_shp: pass
    if csv: pass
    if geopckg: pass

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To export a GeoDataFrame to different file formats, you can use geopandas' built-in methods. Here's how to implement each:

**Shapefile:**
```python
gdf.to_file(folder_out / "filename.shp")
```

**CSV:**
```python
gdf.to_csv(folder_out / "filename.csv", index=False)
```

**GeoPackage:**
```python
gdf.to_file(folder_out / "filename.gpkg", driver="GPKG")
```

For the CSV export, you might want to drop the geometry column first (since CSV can't store geometry) or convert it to WKT format. What would you prefer - include geometry as WKT text, or just export the coordinate columns without the geometry object?

I don't want to drop the geometry column completely. I want to keep the x and y coordinates in a x and y column. How do I do that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can extract the x and y coordinates from the geometry column before exporting to CSV. Here's how:

```python
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y
```

Then when you export to CSV, you can drop just the geometry column:

```python
gdf.drop(columns='geometry').to_csv(folder_out / f"{fn}.csv", index=False)
```

Would you like to add this to your `export_gdf` function?

In [ ]:
#| export
def export_gdf(
    gdf: gpd.GeoDataFrame, # GeoDataFrame to be saved
    fn: str, # Filename of the GeoDataFrame without extension
    folder_out: Path, # Absolute path to folder where files can be saved
    esri_shp: bool=True, # Save GeoDataFrame to Esri shapefile?
    csv: bool=True, # Save GeoDataFrame to comma seperated file?
    geopckg: bool=True, # Save GeoDataFrame to geopackage?
    )->None:
    if esri_shp: gdf.to_file(folder_out / f"{fn}.shp")
    if geopckg: gdf.to_file(folder_out / f"{fn}.gpkg", driver="GPKG")
    if csv:
        gdf['x'] = gdf.geometry.x
        gdf['y'] = gdf.geometry.y
        gdf.drop(columns=['geometry'], inplace=True)
        gdf.to_file(folder_out / f"{fn}.csv", index=False)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()